In [1]:
import numpy as np
import skimage as ski
from scipy.signal import find_peaks, butter, filtfilt
import matplotlib.pyplot as plt

In [4]:
from picamera.array import PiYUVArray, PiRGBArray
from picamera import PiCamera
from scipy.signal import find_peaks, butter, filtfilt
import time

res = (1024, 768)
camera = PiCamera()
camera.sensor_mode = 7
camera.resolution = res
camera.framerate = 120
# b, a = butter(3, 0.02)

rawCapture = PiYUVArray(camera, size=res)
stream = camera.capture_continuous(rawCapture, format="yuv", use_video_port=True)

N = 300
k = 0
t = time.time()
mid = 500

for f in stream:
    I = f.array[:, :, 0]
    rawCapture.truncate(0)
    print("showing image")
    plt.imshow(I)
    time.sleep(5)


AttributeError: 'generator' object has no attribute 'array'

In [ ]:
stream.close()
rawCapture.close()
camera.close()

In [ ]:
# Load the sample image

# I = ski.io.imread("track.jpg")

In [ ]:
plt.imshow(I)

In [ ]:
# Print the image dimensions in pixels

I.shape

In [ ]:
# Conversion to gray
Ibw = ski.color.rgb2gray(I)

In [ ]:
Ibw.shape

In [ ]:
# Since the image has a black line on a white background 
# we invert the image by substracting it from its maximum value.
# We also select a smaller region of interest

ROI = Ibw # Ibw.max() - Ibw[1350:1400, 1000:3000]

In [ ]:
# Print the minimum and the maximum values

ROI.min(), ROI.max()

In [ ]:
# Select the 25th row in the image for line detection
L = ROI[25]

# Plot the intensities across this row
plt.plot(L)

In [ ]:
# Due to the noise in the image the algorithm finds many peaks heigher than 0.5
find_peaks(L, height=0.5)

In [ ]:
# To filter the noise in the image we use a 3rd order Butterworth filter

# Wn = 0.02, the cut-off frequency, acceptable values are from 0 to 1
b, a = butter(3, 0.02)

In [ ]:
# Filter the selected row in the image using the above filter coefficients a, b
Lf = filtfilt(b, a, L)

In [ ]:
# Plot the pixel intensities along the selected row
plt.plot(L, label="raw")
plt.plot(Lf, label="filtered")

plt.legend()

In [ ]:
# Find the two peaks in the filtered data
find_peaks(Lf, height=0.5)

In [ ]:
%%timeit

## Estimate the run time of the line detection algorithm

# Select the 25th row in the image for line detection
L = ROI[25]

# To filter the noise in the image we use a 3rd order Butterworth filter
b, a = butter(3, 0.02)

# Smooth the transitions so we can detect the peaks 
Lf = filtfilt(b, a, L)

# Find peaks which are higher than 0.5
p = find_peaks(Lf, height=0.5)